In [1]:
import os
print(os.getcwd())

e:\nlp_projects\fake_news_classifier\src


In [1]:
from data.load_data import load_data
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
data = load_data(fake='../data/raw/fake.csv', true='../data/raw/true.csv')
X = data[['title', 'text']]
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
tf_idf_vectorizer = TfidfVectorizer(max_features=20000, stop_words='english')
tf_idf_features = tf_idf_vectorizer.fit_transform(X_train['title'] + ' ' + X_train['text'])
print(tf_idf_features.shape)

e:\nlp_projects\fake_news_classifier\src
(35918, 20000)


In [3]:
from models.multinomialNB import train_multinomial_nb
multinomial_nb_model = train_multinomial_nb(tf_idf_features, y_train)

In [4]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = multinomial_nb_model.predict(tf_idf_vectorizer.transform(X_test['title'] + ' ' + X_test['text']))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9346325167037862
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.94      0.94      4733
           1       0.94      0.92      0.93      4247

    accuracy                           0.93      8980
   macro avg       0.93      0.93      0.93      8980
weighted avg       0.93      0.93      0.93      8980



In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from data.preproces import prepross_data
X_train['combined'] = X_train['title'] + ' ' + X_train['text']
X_test['combined'] = X_test['title'] + ' ' + X_test['text']
X_train['tokens'] = X_train['combined'].apply(prepross_data)
print(X_train['tokens'].head())

36335    [turkey, erdogan, says, u, jerusalem, decision...
12384    [breaking, trump, reportedly, considering, ted...
24419    [trump, chief, staff, civil, war, comment, spa...
24740    [saudi, arabia, welcomes, new, u, strategy, to...
27039    [trump, arrives, sicily, ahead, group, seven, ...
Name: tokens, dtype: object


In [7]:
from gensim.models import Word2Vec
word2vec_model = Word2Vec(sentences=X_train['tokens'], vector_size=300, window=5 , min_count=2)

In [8]:
print(word2vec_model.wv['fake'])
print(word2vec_model.wv.most_similar('fake', topn=5))

[-0.23546109  0.44447407  0.68707     0.42577636 -1.3456494  -0.3224576
 -0.39997825 -0.06032753 -1.1606032  -0.19441223 -0.9267706   1.1028668
 -1.4829093  -0.903851    1.1434504  -1.3727868   0.4260347   1.3016876
 -0.00388264 -0.19668102 -0.4511752  -0.65969634 -0.4117543  -0.8183775
  2.2860703  -1.8212067   0.2602375  -0.5076044   0.07074233  0.59413785
  1.6980408  -0.80506563 -0.7932385  -1.3521756  -0.21340019 -0.20118305
  0.63177586 -1.4889787   0.2613961  -1.9791913  -0.10794171  0.4683131
 -1.0115341   0.46442825  0.38932624 -0.8366437  -0.6231545  -0.61348796
 -1.4381984  -1.8805598  -0.5373579  -0.00484727 -0.57589734 -0.02712785
  0.11285156 -1.0680015   1.2501627   0.4958915  -0.34312862  0.7553021
 -1.0359751  -2.445959    1.0683869  -2.6466737  -0.2822519  -0.20684381
 -1.0604156  -0.4353349  -0.32356647 -1.6209958   0.46261033  0.9313239
  1.1387702  -1.2053345   0.01887727 -1.2864392  -0.3693863  -1.4514173
  0.33395725  1.4397235  -1.4279494  -1.6788099   1.3329599

In [9]:
import numpy as np
def doc2vec(tokens,model):
    doc_vectors = [model.wv[token] for token in tokens if token in model.wv]
    return np.mean(doc_vectors,axis=0)

In [3]:
X_test['tokens'] = X_test['combined'].apply(prepross_data)


In [ ]:
X_train['word2vec'] = X_train['tokens'].apply(lambda x: doc2vec(x, word2vec_model))
X_test['word2vec'] = X_test['tokens'].apply(lambda x: doc2vec(x, word2vec_model))

NameError: name 'doc2vec' is not defined

In [11]:
from torch.utils.data import Dataset, DataLoader
import torch
class NewsDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]
    
train_tensor = torch.tensor(X_train['word2vec'].tolist(), dtype=torch.float32)
train_labels = torch.tensor(y_train.values, dtype=torch.float32)
test_tensor = torch.tensor(X_test['word2vec'].tolist(), dtype=torch.float32)
test_labels = torch.tensor(y_test.values, dtype=torch.float32)
train_dataset = NewsDataset(train_tensor, train_labels)
test_dataset = NewsDataset(test_tensor, test_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

C:\Users\PC\AppData\Local\Temp\ipykernel_12340\1426424755.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  train_tensor = torch.tensor(X_train['word2vec'].tolist(), dtype=torch.float32)


In [ ]:
from models.FFN import FFNNModel
from torch import optim
from torch import nn
import importlib 
from models import FFN
importlib.reload(FFN)
from models.train_and_evaluate import train_lstm_model, evaluate_lstm_model
lstm_model = FFNNModel(input_dim=300, hidden_dim=128, output_dim=1)
train_lstm_model(lstm_model, train_loader, num_epochs=100,criterion=nn.BCELoss(), optimizer=optim.SGD(lstm_model.parameters()))

torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size([32, 300])
torch.Size

In [28]:
print(evaluate_lstm_model(lstm_model,test_loader=test_loader))

Test Accuracy: 0.9755
0.9755011135857461


# LSTM Model


In [1]:
from data.preproces import build_vocab
from data import preproces
import importlib 
importlib.reload(preproces)
vocab = build_vocab(X_train['tokens'])

NameError: name 'X_train' is not defined

In [47]:
print(max(vocab.values()))

106489


In [5]:
from torch.utils.data import Dataset, DataLoader
from data import preproces
from data.preproces import build_seq
import torch
import numpy as np
importlib.reload(preproces)
class TextDataSet(Dataset):
    def __init__(self, texts, labels,vocab):
        self.texts = texts
        self.labels = labels
        self.encoded = [build_seq(tokens,vocab,max_len=100) for tokens in texts]
        print(np.max(self.encoded))
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.encoded[idx],dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.float32)

train_dataset = TextDataSet(X_train['tokens'], y_train.values, vocab)
test_dataset = TextDataSet(X_test['tokens'], y_test.values, vocab)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

69691
69666


In [ ]:
from models.lstm import lstm_model
from models import train_and_evaluate
import torch 
from torch import nn
from torch import optim

importlib.reload(train_and_evaluate)
from models.train_and_evaluate import train_lstm_model, evaluate_lstm_model
lstm_model = lstm_model(hidden_dim=128, output_dim=1, vocab_dim=len(vocab), embedded_dim=300,num_layers=2)
print(len(vocab))
train_lstm_model(lstm_model, train_loader, num_epochs=100, criterion=nn.BCELoss(), optimizer=optim.SGD(lstm_model.parameters()))
print(evaluate_lstm_model(lstm_model, test_loader=test_loader))

69693
